In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.0352, train_loss: 2.07760525, train_accuracy: 0.4922, test_Accuracy: 0.2454
Epoch: [ 0] [    1/  468] time: 1.2534, train_loss: 1.99073207, train_accuracy: 0.4766, test_Accuracy: 0.3712
Epoch: [ 0] [    2/  468] time: 1.4943, train_loss: 1.89165688, train_accuracy: 0.5156, test_Accuracy: 0.4862
Epoch: [ 0] [    3/  468] time: 1.7564, train_loss: 1.71332014, train_accuracy: 0.6875, test_Accuracy: 0.5619
Epoch: [ 0] [    4/  468] time: 1.9983, train_loss: 1.66895509, train_accuracy: 0.6328, test_Accuracy: 0.6225
Epoch: [ 0] [    5/  468] time: 2.2483, train_loss: 1.57303858, train_accuracy: 0.7109, test_Accuracy: 0.6808
Epoch: [ 0] [    6/  468] time: 2.4994, train_loss: 1.36856246, train_accuracy: 0.7500, test_Accuracy: 0.7175
Epoch: [ 0] [    7/  468] time: 2.7452, train_loss: 1.30889881, train_accuracy: 0.7266, test_Accuracy: 0.7233
Epoch: [ 0] [    8/  468] time: 3.0012, train_loss: 1.20033681, train_accuracy: 0.7422, test_Accuracy: 0.7341
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 19.7448, train_loss: 0.18857387, train_accuracy: 0.9297, test_Accuracy: 0.9177
Epoch: [ 0] [   76/  468] time: 20.0067, train_loss: 0.22591466, train_accuracy: 0.9375, test_Accuracy: 0.9196
Epoch: [ 0] [   77/  468] time: 20.2273, train_loss: 0.41870493, train_accuracy: 0.8984, test_Accuracy: 0.9215
Epoch: [ 0] [   78/  468] time: 20.4785, train_loss: 0.23376207, train_accuracy: 0.9297, test_Accuracy: 0.9221
Epoch: [ 0] [   79/  468] time: 20.7919, train_loss: 0.21991074, train_accuracy: 0.9375, test_Accuracy: 0.9224
Epoch: [ 0] [   80/  468] time: 21.0446, train_loss: 0.30123270, train_accuracy: 0.9219, test_Accuracy: 0.9225
Epoch: [ 0] [   81/  468] time: 21.2912, train_loss: 0.20620571, train_accuracy: 0.9375, test_Accuracy: 0.9222
Epoch: [ 0] [   82/  468] time: 21.6703, train_loss: 0.22843733, train_accuracy: 0.9375, test_Accuracy: 0.9214
Epoch: [ 0] [   83/  468] time: 21.9207, train_loss: 0.13987195, train_accuracy: 0.9609, test_Accuracy: 0.9217
E

Epoch: [ 0] [  149/  468] time: 38.4418, train_loss: 0.29387414, train_accuracy: 0.9062, test_Accuracy: 0.9321
Epoch: [ 0] [  150/  468] time: 38.6877, train_loss: 0.18016939, train_accuracy: 0.9453, test_Accuracy: 0.9309
Epoch: [ 0] [  151/  468] time: 38.9146, train_loss: 0.19108255, train_accuracy: 0.9297, test_Accuracy: 0.9343
Epoch: [ 0] [  152/  468] time: 39.1443, train_loss: 0.22339429, train_accuracy: 0.9219, test_Accuracy: 0.9345
Epoch: [ 0] [  153/  468] time: 39.4281, train_loss: 0.20655248, train_accuracy: 0.9453, test_Accuracy: 0.9328
Epoch: [ 0] [  154/  468] time: 39.6910, train_loss: 0.19481227, train_accuracy: 0.9453, test_Accuracy: 0.9321
Epoch: [ 0] [  155/  468] time: 39.9182, train_loss: 0.15348300, train_accuracy: 0.9609, test_Accuracy: 0.9315
Epoch: [ 0] [  156/  468] time: 40.1506, train_loss: 0.20440067, train_accuracy: 0.9375, test_Accuracy: 0.9336
Epoch: [ 0] [  157/  468] time: 40.4260, train_loss: 0.20296369, train_accuracy: 0.9297, test_Accuracy: 0.9338
E

Epoch: [ 0] [  223/  468] time: 56.6922, train_loss: 0.13304189, train_accuracy: 0.9531, test_Accuracy: 0.9384
Epoch: [ 0] [  224/  468] time: 56.9685, train_loss: 0.15445733, train_accuracy: 0.9766, test_Accuracy: 0.9427
Epoch: [ 0] [  225/  468] time: 57.2053, train_loss: 0.18654381, train_accuracy: 0.9531, test_Accuracy: 0.9471
Epoch: [ 0] [  226/  468] time: 57.4452, train_loss: 0.13776961, train_accuracy: 0.9297, test_Accuracy: 0.9477
Epoch: [ 0] [  227/  468] time: 57.7076, train_loss: 0.10519689, train_accuracy: 0.9766, test_Accuracy: 0.9472
Epoch: [ 0] [  228/  468] time: 57.9389, train_loss: 0.21535778, train_accuracy: 0.9375, test_Accuracy: 0.9450
Epoch: [ 0] [  229/  468] time: 58.2339, train_loss: 0.18482675, train_accuracy: 0.9609, test_Accuracy: 0.9421
Epoch: [ 0] [  230/  468] time: 58.5554, train_loss: 0.13583481, train_accuracy: 0.9688, test_Accuracy: 0.9421
Epoch: [ 0] [  231/  468] time: 58.8134, train_loss: 0.12672439, train_accuracy: 0.9531, test_Accuracy: 0.9434
E

Epoch: [ 0] [  297/  468] time: 75.2406, train_loss: 0.09777987, train_accuracy: 0.9609, test_Accuracy: 0.9558
Epoch: [ 0] [  298/  468] time: 75.4560, train_loss: 0.16267854, train_accuracy: 0.9609, test_Accuracy: 0.9539
Epoch: [ 0] [  299/  468] time: 75.7058, train_loss: 0.19125184, train_accuracy: 0.9531, test_Accuracy: 0.9557
Epoch: [ 0] [  300/  468] time: 75.9537, train_loss: 0.18367037, train_accuracy: 0.9375, test_Accuracy: 0.9570
Epoch: [ 0] [  301/  468] time: 76.2018, train_loss: 0.14917101, train_accuracy: 0.9609, test_Accuracy: 0.9575
Epoch: [ 0] [  302/  468] time: 76.4199, train_loss: 0.17591219, train_accuracy: 0.9531, test_Accuracy: 0.9569
Epoch: [ 0] [  303/  468] time: 76.6601, train_loss: 0.15250708, train_accuracy: 0.9453, test_Accuracy: 0.9569
Epoch: [ 0] [  304/  468] time: 76.9460, train_loss: 0.10028718, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  305/  468] time: 77.1772, train_loss: 0.13713023, train_accuracy: 0.9453, test_Accuracy: 0.9541
E

Epoch: [ 0] [  371/  468] time: 93.5309, train_loss: 0.16453236, train_accuracy: 0.9453, test_Accuracy: 0.9580
Epoch: [ 0] [  372/  468] time: 93.7887, train_loss: 0.08554219, train_accuracy: 0.9844, test_Accuracy: 0.9578
Epoch: [ 0] [  373/  468] time: 94.0248, train_loss: 0.07586550, train_accuracy: 0.9844, test_Accuracy: 0.9585
Epoch: [ 0] [  374/  468] time: 94.2574, train_loss: 0.06762670, train_accuracy: 0.9766, test_Accuracy: 0.9584
Epoch: [ 0] [  375/  468] time: 94.4850, train_loss: 0.09824596, train_accuracy: 0.9688, test_Accuracy: 0.9592
Epoch: [ 0] [  376/  468] time: 94.7311, train_loss: 0.19323054, train_accuracy: 0.9375, test_Accuracy: 0.9596
Epoch: [ 0] [  377/  468] time: 94.9979, train_loss: 0.12060337, train_accuracy: 0.9766, test_Accuracy: 0.9595
Epoch: [ 0] [  378/  468] time: 95.2438, train_loss: 0.28634158, train_accuracy: 0.9609, test_Accuracy: 0.9588
Epoch: [ 0] [  379/  468] time: 95.5299, train_loss: 0.11609069, train_accuracy: 0.9844, test_Accuracy: 0.9582
E

Epoch: [ 0] [  445/  468] time: 111.9724, train_loss: 0.24320732, train_accuracy: 0.9531, test_Accuracy: 0.9620
Epoch: [ 0] [  446/  468] time: 112.1921, train_loss: 0.09796606, train_accuracy: 0.9766, test_Accuracy: 0.9614
Epoch: [ 0] [  447/  468] time: 112.4586, train_loss: 0.16128968, train_accuracy: 0.9531, test_Accuracy: 0.9615
Epoch: [ 0] [  448/  468] time: 112.6796, train_loss: 0.15023328, train_accuracy: 0.9688, test_Accuracy: 0.9601
Epoch: [ 0] [  449/  468] time: 112.9139, train_loss: 0.14687046, train_accuracy: 0.9297, test_Accuracy: 0.9599
Epoch: [ 0] [  450/  468] time: 113.1580, train_loss: 0.11025532, train_accuracy: 0.9688, test_Accuracy: 0.9597
Epoch: [ 0] [  451/  468] time: 113.3950, train_loss: 0.09442145, train_accuracy: 0.9609, test_Accuracy: 0.9611
Epoch: [ 0] [  452/  468] time: 113.6286, train_loss: 0.12430377, train_accuracy: 0.9531, test_Accuracy: 0.9612
Epoch: [ 0] [  453/  468] time: 113.9037, train_loss: 0.27528811, train_accuracy: 0.9219, test_Accuracy:

# test accuracy: 96.69%